# 支持向量机

支持向量机（SVM）是个非常强大并且有多种功能的机器学习模型，能够做线性或者非线性的分类，回归，甚至异常值检测。机器学习领域中最为流行的模型之一，是任何学习机器学习的人必备的工具。

__SVM 特别适合应用于复杂但中小规模数据集的分类问题__。

## 线性支持向量机分类

### gap

__SVM 对特征缩放比较敏感__

### 软/硬间隔分类

如果我们严格地规定所有的数据都不在“街道（gap）”上，都在正确地两边，称为 __硬间隔分类__。硬间隔分类有两个问题：

* 第一，只对线性可分的数据起作用；
* 第二，对异常点敏感。

为了避免上述的问题，我们更倾向于使用更加软性的模型。目的在保持“街道”尽可能大和避免间隔违规（例如：数据点出现在“街道”中央或者甚至在错误的一边）之间找到一个良好的平衡。这就是 __软间隔分类__。

> 在 Scikit-Learn 库的 SVM 类，可以用 __`C`__ 这个超参数（惩罚系数）来控制这种平衡：__较小的 `C` 会导致更宽的“街道”，但更多的间隔违规__。如果的 SVM 模型过拟合，可以尝试通过减小超参数 `C` 去调整。

### 实例

使用内置的鸢尾花（Iris）数据集，缩放特征，并训练一个线性 SVM 模型（使用 `LinearSVC` 类，超参数 `C=1`，hinge 损失函数）来检测 Virginica 鸢尾花。

> hinge 损失函数：

In [1]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [2]:
iris = datasets.load_iris()

X = iris["data"][:, (2, 3)]  # petal length, petal width 
y = (iris["target"] == 2).astype(np.float64)  # Iris-Virginica 

In [3]:
svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge"))
])

In [5]:
svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linear_svc',
                 LinearSVC(C=1, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=1000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

In [6]:
svm_clf.predict([[5.5, 1.7]])

array([1.])

可以在 SVC 类，使用 `SVC(kernel="linear", C=1)`  ，但是它比较慢，尤其在较大的训练集上，所以一般不被推荐。

另一个选择是使用 `SGDClassifier` 类，即 `SGDClassifier(loss="hinge", alpha=1/(m*C))`。它应用了随机梯度下降来训练一个线性 SVM 分类器。尽管它不会和 `LinearSVC` 一样快速收敛，但是对于处理那些不适合放在内存的大数据集是非常有用的，或者处理在线分类任务同样有用。

# 非线性支持向量机分类

尽管线性 SVM 分类器在许多案例上表现得出乎意料的好，但是很多数据集并不是线性可分的。一种处理非线性数据集方法是增加更多的特征，例如多项式特征；在某些情况下可以变成线性可分的数据。

__方案一：__ 添加新的多项式特征

通过 Scikit-Learn，创建一个流水线（Pipeline）去包含多项式特征（`PolynomialFeatures`）变换，然后使用 `StandardScaler` 和`LinearSVC`。

```python
polynomial_svm_clf = Pipeline((
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss= "hinge"))
))

polynomial_svm_clf.fit(X, y)
```

__方案二：__ 使用多项式核（poly）

添加多项式特征很容易实现，不仅仅在 SVM，在各种机器学习算法都有不错的表现，__但是低次数的多项式不能处理非常复杂的数据集，而高次数的多项式却产生了大量的特征，会使模型变得慢__。

当使用SVM时，可以运用一个被称为“核技巧”（kernel trick）的神奇数学技巧。__它可以取得就像添加了许多多项式，甚至有高次数的多项式，一样好的结果。所以不会大量特征导致的组合爆炸，因为你并没有增加任何特征。__

```python
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline((
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1 ,C=5))
))

poly_kernel_svm_clf.fit(X, y)
```

> __如果你的模型过拟合，可以减小多项式核的阶数。相反的，如果是欠拟合，你可以尝试增大它__。超参数 `coef0` 控制了高阶多项式与低阶多项式对模型的影响。

> 通用的方法是使用网格搜索去找到最优超参数。

## 高斯核（高斯径向基函数RBF）

```python
rbf_kernel_svm_clf = Pipeline((
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
))

rbf_kernel_svm_clf.fit(X, y)
```

如图所示：

![Image00109.jpg](./images/Image00109.jpg)

这么多可供选择的核函数，如何决定使用哪一个？一般来说，__应该先尝试线性核函数__（记住 `LinearSVC` 比 `SVC(kernel="linear")` 要快得多），尤其是当训练集很大或者有大量的特征的情况下。__如果训练集不太大，你也可以尝试高斯径向基核（Gaussian RBF Kernel），它在大多数情况下都很有效。__

## 计算复杂度

`LinearSVC` 类基于 `liblinear` 库，它实现了线性 SVM 的优化算法。并不支持核技巧，但是它样本和特征的数量几乎是线性的：训练时间复杂度大约为 `O(m × n)`。

如果要非常高的精度，这个算法需要花费更多时间。__这是由容差值超参数 `ϵ`（在 Scikit-learn 称为 `tol`）控制的。大多数分类任务中，使用默认容差值的效果是已经可以满足一般要求。

SVC 类基于 `libsvm` 库，它实现了支持核技巧的算法。训练时间复杂度通常介于 `O(m^2 × n)` 和 `O(m^3 × n)` 之间。不幸的是，这意味着 __当训练样本变大时，它将变得极其慢__（例如，成千上万个样本）。这个算法对于复杂但小型或中等数量的数据集表现是完美的。然而，它能对特征数量很好的缩放，尤其对稀疏特征来说（sparse features）（即每个样本都有一些非零特征）。在这个情况下，算法对每个样本的非零特征的平均数量进行大概的缩放。

# SVM 回归

SVM 算法应用广泛：不仅仅支持线性和非线性的分类任务，还支持线性和非线性的回归任务。技巧在于逆转我们的目标：__限制间隔违规的情况下，不是试图在两个类别之间找到尽可能大的“街道”（即间隔）。SVM 回归任务是限制间隔违规情况下，尽量放置更多的样本在“街道”上__。“街道”的宽度由超参数 `ϵ` 控制。

> __添加更多的数据样本在间隔之内并不会影响模型的预测，因此，这个模型认为是不敏感的（ϵ-insensitive）__。

可以使用 Scikit-Learn 的 `LinearSVR` 类去实现线性 SVM 回归。

```python
from sklearn.svm import LinearSVR
# 此处应有标准化
svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)
```

> 使用了 Scikit-Learn 的 `SVR` 类（支持核技巧）。在回归任务上，`SVR` 类和 `SVC` 类是一样的，并且 `LinearSVR` 是和 `LinearSVC` 等价。`LinearSVR` 类和训练集的大小成线性（就像 `LinearSVC` 类），当训练集变大，`SVR` 会变的很慢（就像 `SVC` 类）

# 背后机制

hinge 损失

函数 `max(0, 1–t)` 被称为 Hinge 损失函数（如下）。当 `t≥1` 时，Hinge 值为 0。如果 `t<1`,它的导数（斜率）为 -1，若 `t>1`，则等于0。

![Image00251.jpg](./images/Image00251.jpg)